Import needed libraries

In [2]:
import random
import numpy as np

### Data Preprocessing

This function gets an address which belongs to testcase and reads it and creates flow and distance matrices

In [3]:
def read_data(address):
    with open (address, 'r') as f:
        # read n
        n = int(f.readline().strip())
        f.readline()
        D = []

        # read distance matrix
        for _ in range(n):
            rowint = []
            row = f.readline().strip()
            num = ""
            for c in row:
                if c != " ":
                    num += c
                elif num == "":
                    continue
                else:
                    rowint.append(int(num))
                    num = ""
            rowint.append(int(num))
            D.append(rowint)

        f.readline()
        F = []
        # read flow matrix
        for _ in range(n):
            rowint = []
            row = f.readline().strip()
            num = ""
            for c in row:
                if c != " ":
                    num += c
                elif num == "":
                    continue
                else:
                    rowint.append(int(num))
                    num = ""
            rowint.append(int(num))
            F.append(rowint)

    return n, D, F

### Simulated Annealing Algorithm

This Function evaluates the goodness of achieved results based on following equation
$$\phi(\pi)=\sum_{i=1}^{n}\sum_{j=1}^{n}d_{ij}f_{\pi_{i}\pi_{j}}$$

In [4]:
def evaluation_function(state, D, F):
    n = len(state)
    cost = 0
    for i in range(n):
        for j in range(n):
            cost += D[i][j] * F[state[i]][state[j]]
    return cost

This function finds 3 neighbors for given state by swapping 2 random indexes and then chooses the best one of them and returns it

In [5]:
def find_neighbor(state, D, F):  
    len_cuts = len(state)
    s1 = state.copy()
    s2 = state.copy()
    s3 = state.copy()

    i = random.randint(0, len_cuts-1)
    j = random.randint(0, len_cuts-1)
    i1 = random.randint(0, len_cuts-1)
    j1 = random.randint(0, len_cuts-1)
    i2 = random.randint(0, len_cuts-1)
    j2 = random.randint(0, len_cuts-1)

    s1[i], s1[j] = s1[j], s1[i]
    s2[i1], s2[j1] = s2[j1], s2[i1]
    s3[i2], s3[j2] = s3[j2], s3[i2]

    c1 = evaluation_function(s1, D, F)
    c2 = evaluation_function(s2, D, F)
    c3 = evaluation_function(s3, D, F)

    if c1 > c2 :
      if c1 > c3 :
        return s1
      else:
        return s3
    return s2


This function chooses the state that should be replaced by surrent state. If new state has a better cost, repleces it, else it replace new state with some probability: 
$$\theta\gt exp(\frac{-\Delta}{T\times \beta})$$

In [45]:
def acceptance_probability(current_value, new_value, temprature, beta=400):
    delta = (new_value - current_value)/current_value

    if delta < 0:           # if the value of new state is better than current state, exchange them
        return 1

    r = np.random.rand()     # else: generate a random number between 0, 1
    p = np.exp(-delta/temprature*beta)  # calculate acceptance probability
    
    if (p > r):        # accept new state with calculated probability 
        return 2 
    return 0


This function decreases temprature
$$T=T\times \alpha$$

In [12]:
def cooling_function(current_temp):
    alpha = 0.99
    new_temp = current_temp * alpha
    return new_temp

simulated annealing's main algorithm

In [34]:
def simulated_annealing(permuterm, temprature, TL, iteration, D, F, expected):
    current_state = permuterm.copy()
    random.shuffle(current_state)          # generate a random initial state
    current_value = evaluation_function(current_state, D, F)           # find the value of current state
    best_result = current_state[:]         # save the best result 
    best_cost = np.inf     # save the best cost
    num_iteration = 0

    # search for result until the temprature reaches the temperature_low or we achieve the expected result
    while temprature > TL and best_cost > expected:
        num_iteration = 0
        while num_iteration<iteration:    # for each temperature, search for result n times
            num_iteration += 1
            neighbor       = find_neighbor(current_state, D, F)             # search for a neighbor and find its cost
            neighbor_value = evaluation_function(neighbor, D, F)

            p = acceptance_probability(current_value, neighbor_value, temprature)  # the result clarifies that which state should be the current state
            if p == 1:   # p=1 : exchange with new state, new state has better fitness
                current_state = neighbor.copy()
                current_value = neighbor_value
                best_result = neighbor[:]
                best_cost =  current_value

            elif p == 2:     # exchange with new state, new state is worse than current state
                current_state = neighbor.copy()
            # p=0, don't exchange states
            

        temprature = cooling_function(temprature)   # lower the temprature

    return best_cost, best_result


Test chr12a

The best achieved result is 9552

In [36]:
n, D, F = read_data("testcases/chr12a.dat")
initial_state = [i for i in range(n)]
result1c, result1 = simulated_annealing(initial_state, 1, 0.01, 100000, D, F, 9560)
print("Best result: ", result1)
print("Best cost: ", result1c)

Best result: [6, 4, 11, 1, 0, 2, 8, 10, 9, 5, 7, 3]
Best cost: 9552


Test esc32a

In [37]:
n, D, F = read_data("testcases/esc32a.dat")
initial_state = [i for i in range(n)]
result2c, result2 = simulated_annealing(initial_state, 1, 0.01, 1000, D, F, 170)
print("Best result: ", result2)
print("Best cost: ", result2c)

Best result:  [8, 10, 9, 5, 12, 4, 11, 30, 13, 17, 16, 23, 0, 14, 15, 31, 20, 7, 18, 29, 27, 6, 26, 19, 28, 21, 22, 24, 2, 1, 3, 25]
Best cost:  168


Test nug20

In [43]:
n, D, F = read_data("testcases/nug20.dat")
initial_state = [i for i in range(n)]
result3c, result3 = simulated_annealing(initial_state, 1, 0.01, 2000, D, F, 2590)
print("Best result: ", result3)
print("Best cost: ", result3c)

Best result:  [8, 2, 9, 13, 17, 15, 10, 11, 1, 3, 12, 7, 19, 14, 18, 5, 0, 6, 4, 16]
Best cost:  2570


Test tai30a

In [47]:
n, D, F = read_data("testcases/tai30a.dat")
initial_state = [i for i in range(n)]
result4c, result4 = simulated_annealing(initial_state, 1, 0.01, 1000, D, F, 1890900)
print("Best result: ", result4)
print("Best cost: ", result4c)

Best result:  [14, 29, 17, 25, 23, 28, 2, 19, 10, 21, 20, 6, 0, 9, 4, 27, 1, 15, 16, 5, 8, 3, 26, 12, 7, 13, 22, 11, 24, 18]
Best cost:  1890518


Test lipa50a

In [52]:
n, D, F = read_data("testcases/lipa50a.dat")
initial_state = [i for i in range(n)]
result5c, result5 = simulated_annealing(initial_state, 1, 0.01, 1000, D, F, 63050)
print("Best result: ", result5)
print("Best cost: ", result5c)

Best result:  [14, 6, 30, 44, 0, 12, 10, 37, 22, 18, 13, 8, 28, 48, 47, 32, 49, 1, 43, 5, 4, 31, 41, 15, 23, 45, 2, 9, 20, 17, 46, 7, 40, 36, 21, 38, 11, 3, 29, 26, 27, 24, 39, 34, 35, 33, 42, 16, 19, 25]
Best cost:  62990
